# Image Classification for Coffee or Donuts

Is the given image a coffee photo, or a donut?
In this demo notebook we build a simple deep neural network to classify an image as one of the following:
- coffee
- mug
- donut


## Install Prerequisites

We use Keras/Tensorflow to build the classification model, and visualize the process with matplotlib.

In [ ]:
# Import required libraries
from botocore.client import Config
import ibm_boto3
!pip install tensorflow==1.15
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

## Read The Data

Here we build simple wrapper functions to read in the data from our cloud object storage buckets and extract it. First, we need the credentials from cloud object store. 

Open the data side bar (0100). Click on the dropdown for "coffee-donuts-segregated.zip" file and insert "credentials"

In [ ]:
# open the data side bar (0100). Click on the dropdown for "coffee-donuts-segregated.zip" file 
# and insert "credentials"


In [ ]:
credentials =  # paste the credentials variable from the last cell. This is something like credentials_number

A download function from IBM Cloud

In [ ]:
def download_file_cos(credentials, local_file_name, key): 
    '''
    Wrapper function to download a file from cloud object storage using the
    credential dict provided and loading it into memory
    '''
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [ ]:
# change credentials_1 to the variable that was assigned when insertering credentials
download_file_cos(credentials,'coffee-donuts-segregated.zip','coffee-donuts-segregated.zip')

In [ ]:
from io import BytesIO
import zipfile

    
with zipfile.ZipFile('coffee-donuts-segregated.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [ ]:
#you just downloaded some pictures in the local runtime that we will use the train our model
!ls -ltrh data

## Build the Model

We start with a [MobileNetV2](https://arxiv.org/abs/1801.04381) architecture as the backbone [pretrained feature extractor](https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet). We then add a couple of dense layers and a softmax layer to perfom the classification. We freeze the MobileNetV2 backbone with weights trained on ImageNet dataset and only train the dense layers and softmax layer that we have added.

In [ ]:
base_model=tf.keras.applications.MobileNetV2(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(512,activation='relu')(x) #dense layer 1
x=tf.keras.layers.Dense(256,activation='relu')(x) #dense layer 2
preds=tf.keras.layers.Dense(3,activation='softmax')(x) #final layer with softmax activation

model=tf.keras.Model(inputs=base_model.input,outputs=preds)


In [ ]:
#Freeze layers from MobileNetV2 backbone (not to be trained)
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
#Prepare the training dataset as a data generator object
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('data',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=10,
                                                 class_mode='categorical',
                                                 shuffle=True)

### Using Adam, categorical_crossentropy and accuracy as optimization method, loss function and metrics, respectively

In [ ]:
# Build the model
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

## Train the model

In [ ]:
from tensorflow import set_random_seed
set_random_seed(2)
step_size_train=5
log_file = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=4)

## Figure of Training Loss and Accuracy

In [ ]:
# Model accuracy and loss vs epoch
plt.plot(log_file.history['acc'], '-bo', label="train_accuracy")
plt.plot(log_file.history['loss'], '-r*', label="train_loss")
plt.title('Training Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch #')
plt.legend(loc='center right')
plt.show()

## Model Performance

Here we perform inference on some sample data points to determine the performance of the model

In [ ]:
# Mapping labels 
label_map = (train_generator.class_indices)

In [ ]:
label_map

In [ ]:
# Creating a sample inference function
def prediction(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    preds = model.predict(x)
    #print('Predictions', preds)
    
    for pred, value in label_map.items():    
        if value == np.argmax(preds):
            print('Predicted class is:', pred)
            print('With a confidence score of: ', np.max(preds))
    
    return np.argmax(preds)

In [ ]:
#change credentials_1 to the variable that was assigned when insertering credentials
download_file_cos(credentials,'Coffee.jpg','Coffee.jpg')

In [ ]:
#Opening first image
image = Image.open("Coffee.jpg")
image

In [ ]:
#performing inference on above image
prediction('Coffee.jpg', model)

In [ ]:
#Opening second image
#change credentials_1 to the variable that was assigned when insertering credentials
download_file_cos(credentials,'Mug.jpg','Mug.jpg')
image = Image.open("Mug.jpg")
image

In [ ]:
prediction('Mug.jpg', model)

In [ ]:
#Opening third image
#change credentials_1 to the variable that was assigned when insertering credentials
download_file_cos(credentials,'Donut.jpg','Donut.jpg')
image = Image.open("Donut.jpg")
image

In [ ]:
prediction('Donut.jpg', model)

## Thank you for attending!